In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [52]:
SBA = pd.read_csv("SBA/SBAnational.csv", low_memory=False)
SBA.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [53]:
predictable = SBA[SBA["MIS_Status"].notnull()]
approve_loan = pd.get_dummies(predictable["MIS_Status"], drop_first=True).rename(columns={"P I F": "Approve"})
predictable = pd.concat([predictable,approve_loan], axis=1)

In [54]:
train, test = train_test_split(predictable, test_size=.25, random_state=42)

In [55]:
train["NAICS"]

637780    337110
254877    541611
620299    561740
812761         0
732714    238210
           ...  
260306    812199
367065    721110
132743    621111
672813    332994
122738         0
Name: NAICS, Length: 672875, dtype: int64

NAICS

In [103]:
def naics_classifier(mat):
    rows, cols = mat.shape
    assert cols == 1
    def encode(x):
        mapping = {
            "0": np.nan,
            "31": "31-33",
            "32": "31-33",
            "33": "31-33",
            "44": "44-45",
            "48": "48-49",
            "49": "48-49",
        }
        x = str(x)
        x = "0" if x[0] == "0" else x[:2]
        return mapping.get(x, x)

    if hasattr(mat, "iloc"):
        return mat.iloc[:, 0].apply(encode).to_frame()
    else:
        return mat[:, 0].apply(encode).reshape((rows, 1))

naics_pipeline = Pipeline(
    steps=[
        ("naics_class", FunctionTransformer(naics_classifier)), 
        ("encoding", OneHotEncoder(sparse=False))
        ]
)


tf_pipe = ColumnTransformer(
    transformers=[
    ("NACIS", naics_pipeline, ["NAICS"]),
])
tf_pipe.fit_transform(train).shape



(672875, 22)

State

In [57]:
tf_pipe = ColumnTransformer(
    transformers=[
    ("State", OneHotEncoder(sparse=False), ["BankState"]),
])
tf_pipe.fit_transform(train).shape


(672875, 56)

Urban RUral

In [58]:
tf_pipe = ColumnTransformer(
    transformers=[
    ("State", OneHotEncoder(sparse=False), ["UrbanRural"]),
])
tf_pipe.fit_transform(train)


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

Backed by Real Estate

In [101]:
def backed_by_real_estate(mat):
    rows, cols = mat.shape
    assert cols == 1
    def encode(x):
        return 0 if x < 240 else 1

    if hasattr(mat, "iloc"):
        return mat.iloc[:, 0].apply(encode).to_frame()
    else:
        return mat[:, 0].apply(encode).reshape((rows, 1))

tf_pipe = ColumnTransformer(
    transformers=[
    ("RealEstate", FunctionTransformer(backed_by_real_estate), ["Term"]),
])
tf_pipe.fit_transform(train).shape



(672875, 1)

SBA Ratio

In [60]:
def SBA_portion(mat):
    currency_cleaning = lambda x: int(float(x[1:].replace(",", "")))
    rows, cols = mat.shape
    assert cols == 2  # if we don't have 2 columns, things are unexpected
    
    if hasattr(mat, 'iloc'):
        mat.iloc[:, 0] = mat.iloc[:, 0].apply(currency_cleaning).astype(int)
        mat.iloc[:, 1] = mat.iloc[:, 1].apply(currency_cleaning).astype(int)
        res = mat.iloc[:, 0] / mat.iloc[:, 1]
        return res.to_frame()
    else:
        mat[:, 0] = mat[:, 0].apply(currency_cleaning).astype(int)
        mat[:, 1] = mat[:, 1].apply(currency_cleaning).astype(int)
        res = mat[:, 0] / mat[:, 1]
        return res.reshape((rows, 1))

sba_portion = FunctionTransformer(SBA_portion)

tf_pipe = ColumnTransformer(
    transformers=[
    ('SBA_portion', sba_portion, ['SBA_Appv', 'GrAppv']),
])

tf_pipe.fit_transform(train).shape

(672875, 1)

New or Old Business

In [87]:
train["NewExist"].value_counts()


1.0    482567
2.0    189439
0.0       770
Name: NewExist, dtype: int64

In [86]:
train["NewExist"].unique()

array([ 1.,  2.,  0., nan])

In [97]:
tf_pipe = ColumnTransformer(
    transformers=[
    ('New', OneHotEncoder(sparse=False, handle_unknown="ignore"), ['NewExist']),
])

tf_pipe.fit_transform(train)

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

Nan and 0 value for new exists are few so it can be dropped.

Recession

In [105]:
def loan_active_during_recession(mat):
    rows, cols = mat.shape
    assert cols == 2  # if we don't have 2 columns, things are unexpected
    
    if hasattr(mat, 'iloc'):
        rec_date = pd.to_datetime(mat.iloc[:, 0]) + pd.to_timedelta(mat.iloc[:, 1]*30, unit="D")
        recession = pd.Series(0, index=mat.index)
        recession[(rec_date >= pd.to_datetime("2007-12-01")) & (rec_date <= pd.to_datetime("2009-06-30"))] = 1 
        return recession.to_frame()
    else:
        rec_date = pd.to_datetime(mat[:, 0]) + pd.to_timedelta(mat[:, 1]*30, unit="D")
        recession = pd.Series(0, index=mat.index)
        recession[(rec_date >= pd.to_datetime("2007-12-01")) & (rec_date <= pd.to_datetime("2009-06-30"))] = 1 
        return recession.to_frame()

recession_active = FunctionTransformer(loan_active_during_recession)

tf_pipe = ColumnTransformer(
    transformers=[
    ('Recession', recession_active, ['DisbursementDate', 'Term']),
])
tf_pipe.fit_transform(train)





array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)